In [20]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import asyncio
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

In [21]:
model_client = OpenAIChatCompletionClient(
    model = "gemini-1.5-flash-8b",
    api_key = api_key,
)

In [22]:
import json
with open("team_state.json", "r") as f:
    team_state = json.load(f)

team_state

{'type': 'TeamState',
 'version': '1.0.0',
 'agent_states': {'WriterAgent': {'type': 'ChatAgentContainerState',
   'version': '1.0.0',
   'agent_state': {'type': 'AssistantAgentState',
    'version': '1.0.0',
    'llm_context': {'messages': [{'content': "Write a crime novel on the topic of 'The Great Heist' and then make a movie script on it.",
       'source': 'user',
       'type': 'UserMessage'},
      {'content': "**Novel:**\n\nThe vault's empty.  Not a trace.  But whispers trail,  a network of deceit.  Only one could pull it off:  the phantom thief, the Maestro.\n\n**Movie Script:**\n\n**(Opening scene):**  Empty vault.  Security footage flickers.  A single, masked figure vanishes.  **(Voiceover):**  The Great Heist.  Years later, a detective unearths clues.  **(Cut to interrogation):**  Suspect denies all.  **(Cut to flashback):**  Maestro's past, a web of connections.  **(Final scene):**  A face emerges from the shadows.  The Maestro.\n\n\n",
       'thought': None,
       'sour

In [23]:
writer = AssistantAgent(
    name="WriterAgent",
    description="A simple assistant agent.",
    model_client=model_client,
    system_message="You are best contempoary writer who specialises in writhing articles and crime novels. generate articles and novels only in 20-30 words maximum.",
)

director = AssistantAgent(
    name="DirectorAgent",
    description="A simple assistant agent.",
    model_client=model_client,
    system_message="You are a Directer agent who is expertise in making movies and documentaries based on crime novels. generate movie scripts and documentaries only in 20-20 words maximum.",
)

from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination

termination = MaxMessageTermination(max_messages=2)

new_team = RoundRobinGroupChat(
    participants=[writer, director],
    termination_condition= termination,
)    

In [24]:
await new_team.load_state(team_state)

In [25]:
from autogen_agentchat.ui import Console
stream = new_team.run_stream(task="What was the last task you worked on?")
await Console(stream)

---------- TextMessage (user) ----------
What was the last task you worked on?
---------- TextMessage (DirectorAgent) ----------
I was working on generating a movie script and novel outline, focusing on a crime novel and movie script about "The Great Heist."



TaskResult(messages=[TextMessage(id='3a036c52-883c-4bdc-88bc-32712c7fc203', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 13, 1, 29, 53, 211310, tzinfo=datetime.timezone.utc), content='What was the last task you worked on?', type='TextMessage'), TextMessage(id='f239c951-083b-4981-bece-3a371af2b11c', source='DirectorAgent', models_usage=RequestUsage(prompt_tokens=209, completion_tokens=27), metadata={}, created_at=datetime.datetime(2025, 7, 13, 1, 29, 55, 17025, tzinfo=datetime.timezone.utc), content='I was working on generating a movie script and novel outline, focusing on a crime novel and movie script about "The Great Heist."\n', type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')